### Imports

In [26]:
import torch
import os
import re

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
from natsort import natsorted
from torchvision import models

In [27]:
from google.colab import drive
BASE_FOLDER = os.getcwd()
print(BASE_FOLDER)
drive.mount(os.path.join(BASE_FOLDER, "drive"))

/content
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
import zipfile
zipf = zipfile.ZipFile(os.path.join(BASE_FOLDER, "drive", "MyDrive", "data.zip"))
zipf.extractall(os.path.join(BASE_FOLDER, "data"))
zipf.close()

### Creating the pytorch dataset

In [38]:
class LymphomaDataset(Dataset):
    def __init__(self, image_folder, label_folder, use_augmented, aug_img_folder, aug_label_folder, transform=None):
        self.image_folder = image_folder
        self.label_folder = label_folder
        self.transform = transform
        self.using_aug = use_augmented
        self.aug_img_folder = aug_img_folder
        self.aug_label_folder = aug_label_folder

        self.image_paths = natsorted([os.path.join(image_folder, f) for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))])
        if self.using_aug:
          self.aug_image_paths = natsorted([os.path.join(aug_img_folder, f) for f in os.listdir(aug_img_folder) if os.path.isfile(os.path.join(aug_img_folder, f))])
          self.all_image_paths = self.image_paths + self.aug_image_paths

        self.label_paths = natsorted([os.path.join(label_folder, f) for f in os.listdir(label_folder) if os.path.isfile(os.path.join(label_folder, f))])
        if self.using_aug:
          self.aug_label_paths = natsorted([os.path.join(aug_label_folder, f) for f in os.listdir(aug_label_folder) if os.path.isfile(os.path.join(aug_label_folder, f))])
          self.all_label_paths = self.label_paths + self.aug_label_paths

          print(self.all_label_paths)


    def __len__(self):
        return len(self.all_image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.all_image_paths[idx]).convert("RGB")
        label = int(self.read_label_file(self.all_label_paths[idx]))

        if self.transform:
            image = self.transform(image)

        return image, label

    def read_label_file(self, label_path):
        # Implement logic to read labels from label files
        # For example, if labels are in text files, you can use:
        with open(label_path, 'r') as file:
            label = file.read().strip()  # Adjust based on your label format
        return label

# Replace 'your_image_folder' and 'your_label_folder' with the actual paths
base_folder = os.getcwd()
image_folder = os.path.join(base_folder, "data", "images")
augmented_image_folder = os.path.join(base_folder, "data", "augmented_images")
label_folder = os.path.join(base_folder, "data", "labels")
augmented_label_folder = os.path.join(base_folder, "data", "augmented_labels")

#TRY THE ROTATION OF THE IMAGES
#from PIL import Image
#import os

# Define the rotation angle
#angle = 45  # Rotate by a specific angle

# Create a new folder for rotated images if it doesn't exist
#output_folder_name = f"rotated_images_{angle}"
#output_folder_path = os.path.join(base_folder, "data/images", output_folder_name)
#if not os.path.exists(output_folder_path):
#    os.makedirs(output_folder_path)
#else:
#    os.system(f"rm {output_folder_path}/*")
#
#
#
#
#for image_name in os.listdir(image_folder):
#    if image_name.lower().endswith('.png'):
#        # Load the image
#        img_path = os.path.join(image_folder, image_name)
#        img = Image.open(img_path)

        # Rotate the image without cropping (expand = True)
        # If i use expand=True the image change is size for be view correctly.
        # Else if expand=False the #pixels remains the original and the image's corners are cutted
        # CHOOSE WHAT TO DO
#        rotated_img = img.rotate(angle, expand=True, fillcolor=(0, 0, 0))

        # Define the path to save the rotated image
#       save_path = os.path.join(output_folder_path, f"rotate{angle}_{image_name}")

        # Save the rotated image
#        rotated_img.save(save_path)

#        print(f"Rotated image saved to: {save_path}")

# Define transformation (optional, you can customize it based on your needs)
transform = transforms.Compose([
    transforms.Resize((500, 500)),
    transforms.ToTensor(),
])

# Instantiate the custom dataset
dataset = LymphomaDataset(image_folder, label_folder, True, augmented_image_folder, augmented_label_folder, transform=transform)

batch_size = 256

# Create DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Access a sample from the dataset
sample_index = 200
sample_image, sample_label = dataset[sample_index]

# Print information about the sample
print(f"Sample {sample_index} - Image: {sample_image.size()}, Label: {sample_label}")


['/content/data/labels/0.txt', '/content/data/labels/1.txt', '/content/data/labels/2.txt', '/content/data/labels/3.txt', '/content/data/labels/4.txt', '/content/data/labels/5.txt', '/content/data/labels/6.txt', '/content/data/labels/7.txt', '/content/data/labels/8.txt', '/content/data/labels/9.txt', '/content/data/labels/10.txt', '/content/data/labels/11.txt', '/content/data/labels/12.txt', '/content/data/labels/13.txt', '/content/data/labels/14.txt', '/content/data/labels/15.txt', '/content/data/labels/16.txt', '/content/data/labels/17.txt', '/content/data/labels/18.txt', '/content/data/labels/19.txt', '/content/data/labels/20.txt', '/content/data/labels/21.txt', '/content/data/labels/22.txt', '/content/data/labels/23.txt', '/content/data/labels/24.txt', '/content/data/labels/25.txt', '/content/data/labels/26.txt', '/content/data/labels/27.txt', '/content/data/labels/28.txt', '/content/data/labels/29.txt', '/content/data/labels/30.txt', '/content/data/labels/31.txt', '/content/data/la

### Creating the AlexNet

In [31]:
class AlexNet(nn.Module):
    def __init__(self, num_classes: int = 1000, dropout: float = 0.5) -> None:
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
alexnet = AlexNet(num_classes = 3)

### Training loop

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet.parameters(), lr=0.001, momentum=0.9)

num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

alexnet.to(device) # alexnet to the device, gpu if available

for epoch in range(num_epochs):
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    # Set the model to training mode
    alexnet.train()

    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = alexnet(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Update running statistics
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    # Calculate training accuracy and loss for the epoch
    train_accuracy = correct_predictions / total_samples
    average_loss = running_loss / len(dataloader)

    print(f'Training - Epoch: {epoch + 1}, Loss: {average_loss:.4f}, Accuracy: {train_accuracy:.4f}')



Training - Epoch: 1, Loss: 1.0981, Accuracy: 0.3387
Training - Epoch: 2, Loss: 1.0982, Accuracy: 0.3387
